In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
import gensim as gs
import nltk.data
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import re
from collections import Counter
import scipy as scp
import matplotlib.cm as cm
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn import decomposition, pipeline, metrics, grid_search
from sklearn import cross_validation
from sklearn.preprocessing import PolynomialFeatures
import itertools
from sklearn.cross_validation import StratifiedKFold
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, ElasticNet 
import sys
import os
from sklearn.naive_bayes import GaussianNB
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import pairwise_distances
from HTMLParser import HTMLParser
import pickle
from collections import Counter
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

In [13]:
class RegressionClassifier:

    regression_model = None
    n = None
    values = None
    quantiles = None

    def __init__(self, regression_model):
        self.regression_model = regression_model

    def fit(self, X, y):
        self.values = sorted(set(y.tolist()))
        self.n = len(set(y.tolist()))
        self.quantiles = (np.bincount(y)/float(y.shape[0]))[self.values]

    def gaussian(self, x, mu, sig):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

    def predict(self, X, mean_pow=2, gaus_pow=1):
        y_pred = self.regression_model.predict(X)
        t = np.percentile(y_pred, np.cumsum(self.quantiles)*100)

        def q_order(x):
            for i in range(t.shape[0]):
                if x <= t[i]:
                    return i

        return np.array(map(q_order, y_pred.tolist()), dtype=np.int) + 1


def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)


df = pd.read_pickle('./../data/df_basfe.pkl')

# kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

# idx_row_train = np.where(np.array((~df.median_relevance.isnull()).tolist()))[0]
# idx_row_test = np.where(np.array((df.median_relevance.isnull()).tolist()))[0]

# y_train_all = df['median_relevance'].values[idx_row_train].astype(int)

# rel_var = df['relevance_variance'].values[idx_row_train]

# data = np.load('./../data/tfidf_svd.npz')
# data = dict(map(lambda k: ((int(k.split(' ')[0]), int(k.split(' ')[1])), data[k]), data.keys()))

# path = './../data/w2v_models/'
# w2v_models = {}
# for name in os.listdir(path):
#     w2v_models[name] = Word2Vec.load(os.path.join(path, name))
    

# # number of Q in T
# df['nqt'] = map(lambda t: len(t[0].intersection(t[1])), 
#                 zip(map(lambda s: set(s.split()), df['qcnss'].values), 
#                     map(lambda s: set(s.split()), df['tcnss'].values)))

# predictors = df.columns[list(set(range(5, 10)).union([17, 18]).union(range(27, len(df.columns))))]
# X_base = df[predictors].as_matrix()

idx_row_train = np.where(np.array((~df.median_relevance.isnull()).tolist()))[0]
idx_row_test = np.where(np.array((df.median_relevance.isnull()).tolist()))[0]

y_train_all = df['median_relevance'].values[idx_row_train].astype(int)


# def qdist(q):
#     v = df.loc[(df['qcnss'] == q) & (~df['median_relevance'].isnull()), 
#                'median_relevance'].values.astype(int).tolist()
#     v.extend([1, 2, 3, 4])
#     v = np.bincount(v)[1:] - 1
#     return v / float(v.sum())

# qdist_dict = dict(map(lambda q: (q, qdist(q)), set(df['qcnss'].values)))
# q_rel_prior = np.array(map(lambda q: qdist_dict[q], df['qcnss'].values.tolist()))
# X_base = np.hstack((X_base, q_rel_prior))


# X_emp = X_base
# np.savez('./../data/features/X_emp.npz', data=X_emp)

# X_emp_pca = PCA(n_components=100).fit_transform(X_base)
# np.savez('./../data/features/X_emp_pca.npz', data=X_emp_pca)

# data = np.load('./../data/tfidf_svd.npz')
# data = dict(map(lambda k: ((int(k.split(' ')[0]), int(k.split(' ')[1])), data[k]), data.keys()))

# X_lsa_word = data[(3, 300)]

# np.savez('./../data/features/X_lsa_word.npz', data=X_lsa_word)

# w2v = w2v_models['w2v_size100_mincount5_window7.bin']

# Q = np.array(map(lambda s: 
#              reduce(lambda x, y: x + y, 
#                     map(lambda w: w2v[w] if w in w2v else np.zeros(w2v['foot'].shape), s.split()), 
#                     np.zeros(w2v['foot'].shape))/len(s.split()), 
#              df['qcnss'].values))   
# T = np.array(map(lambda s: 
#              reduce(lambda x, y: x + y, 
#                     map(lambda w: w2v[w] if w in w2v else np.zeros(w2v['foot'].shape), s.split()), 
#                     np.zeros(w2v['foot'].shape))/len(s.split()), 
#              df['tcnss'].values))
# X_w2v_local = Q - T

# np.savez('./../data/features/X_w2v_local.npz', data=X_w2v_local)


# txt = map(lambda p: '. '.join(p), zip(df['qcnss'].values, df['tcnss'].values))
# tfv = TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
# X_tfv = tfv.fit_transform(txt)
# svd = TruncatedSVD(n_components=300)
# X_lsa_char = svd.fit_transform(X_tfv)  
# np.savez('./../data/features/X_lsa_char.npz', data=X_lsa_char)

X_emp = np.load('./../data/features/X_emp.npz')['data']
X_emp_pca = np.load('./../data/features/X_emp_pca.npz')['data']
X_lsa_char = np.load('./../data/features/X_lsa_char.npz')['data']
X_lsa_word = np.load('./../data/features/X_lsa_word.npz')['data']
X_w2v_local = np.load('./../data/features/X_w2v_local.npz')['data']

good_predictors = ['p1', 'p2', 'p3', 'p4', 
                   'cos_qt_cnss_tfidf2_svd200',
                   't_len',
                   'q_len',
                   'obm25_3_qt_cnss',
                   'cos_qt_cnss_w2v_size52_mincount1_window3',
                   'cos_min_qd_cnss_tfidf1_svd300',
                   'min_mean_cos_qt_cnss_w2v_size52_mincount1_window3',
                   'd_loglen',
                   'cos_max_qd_cnss_tfidf2_svd200',
                   'cos_mean_qd_cnss_tfidf2_svd100',
                   'cos_qd_cnss_tfidf1_svd100',
                   'cos_max_qd_cnss_w2v_size100_mincount3_window3',
                   'sop_qt_cnss',
                   'jsc_qd_cnss']
good_predictors_idx  = [386, 387, 388, 389, 55, 1, 0, 91, 102, 33, 279, 3, 58, 53, 20, 189, 7, 14]

X_emp_good = X_emp[:, good_predictors_idx]

# X_emp RegressionClassifier GradientBoostingRegressor

In [14]:
X = X_emp
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = GradientBoostingRegressor(loss='ls',
                                      learning_rate=0.1,
                                      n_estimators=500,
                                      max_depth=3,
                                      min_samples_split=2,
                                      min_samples_leaf=1,
                                      subsample=0.9)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.837473277332 0.665697511434
0.836344402653 0.672226702709
0.836799475568 0.678343778241
0.832590051097 0.691597441896
0.83509058069 0.683261783554
0.833725813082 0.667910981464
0.836983927716 0.673236880933
0.838709812198 0.645988716893
0.835184964949 0.674678488027
0.830864184451 0.700294355112
---------------
[0.835376648974, 0.675323664026, 0.00228082125494, 0.0140819671066]


In [15]:
model = GradientBoostingRegressor(loss='ls',
                                  learning_rate=0.1,
                                  n_estimators=500,
                                  max_depth=3,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  subsample=0.9)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [89]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [16]:
np.savez('./../data/features/results/_emp_rc_gbr.npz', 
         score_lb=0.63868191,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp RegressionClassifier RandomForestRegressor

In [ ]:
X = X_emp
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = RandomForestRegressor(n_estimators=1000,
                                  max_depth=45,
                                  max_features='auto',
                                  n_jobs=-1) 
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [ ]:
model = RandomForestRegressor(n_estimators=1000,
                              max_depth=45,
                              max_features='auto',
                              n_jobs=-1) 
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [ ]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [ ]:
np.savez('./../data/features/results/_emp_rc_rfr.npz', 
         score_lb=0.66350144,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_lsa_word SVC

In [113]:
X = StandardScaler().fit_transform(X_lsa_word)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

0.871723078232 0.616944799676
0.873743805447 0.623747926625
0.875316944802 0.57777326624
0.877744742199 0.647443292631
0.874125130925 0.585120635672
0.873536391418 0.584860737542
0.875451830652 0.578920304679
0.872174415725 0.635270903103
0.873628295375 0.604883962332
0.87180896328 0.565992560791


In [115]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [117]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [118]:
np.savez('./../data/features/results/lsaword_svc.npz', 
         score_lb=0.58504,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_lsa_char SVC

In [124]:
X = StandardScaler().fit_transform(X_lsa_char)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

0.955189024571 0.586374989803
0.953555276725 0.619300027646
0.952973546919 0.542499158567
0.95181587709 0.569604116169
0.951827503002 0.613250464269
0.954589565137 0.659811335206
0.952579128057 0.617933990488
0.952056684826 0.622869682577
0.95316306651 0.644803931736
0.950640112684 0.660023888854


In [129]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [130]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [131]:
np.savez('./../data/features/results/lsachar_svc.npz', 
         score_lb=0.58178,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_w2v_local SVC

In [ ]:
X = StandardScaler().fit_transform(X_w2v_local)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

In [9]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [10]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [11]:
np.savez('./../data/features/results/w2vlocal_svc.npz', 
         score_lb=0.62522,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_w2v_local RC on SVR

In [ ]:
X = StandardScaler().fit_transform(X_w2v_local)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print mean

In [14]:
model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [15]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [16]:
np.savez('./../data/features/results/w2vlocal_rc_svr.npz', 
         score_lb=0.63773,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_w2v_local RC on GBR

In [ ]:
X = StandardScaler().fit_transform(X_w2v_local)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = GradientBoostingRegressor(loss='ls',
                                      learning_rate=0.1,
                                      n_estimators=500,
                                      max_depth=3,
                                      min_samples_split=2,
                                      min_samples_leaf=1,
                                      subsample=0.9)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)

In [18]:
model = GradientBoostingRegressor(loss='ls',
                                  learning_rate=0.1,
                                  n_estimators=500,
                                  max_depth=3,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  subsample=0.9)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [19]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [20]:
np.savez('./../data/features/results/w2vlocal_rc_gbr.npz', 
         score_lb=0.58084,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_w2v_local RC on KNR

In [30]:
X = StandardScaler().fit_transform(X_w2v_local)
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = KNeighborsRegressor(n_neighbors=7)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print mean

0.742593807511 0.592799216922
0.733967275356 0.627791879049
0.739380637897 0.612196809104
0.738824334249 0.634972199232
0.73803660758 0.638545516333
0.738832524535 0.629495268768
0.737556833168 0.618567089965
0.736522948781 0.639245022396
0.738189549509 0.612738046256
0.734050523711 0.650682645483
---------------
[ 0.7377955   0.62570337]


In [31]:
model = KNeighborsRegressor(n_neighbors=7)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [32]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [33]:
np.savez('./../data/features/results/w2vlocal_rc_knr.npz', 
         score_lb=0.58902,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_word SVC

In [24]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_word)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.964067891713 0.619497114331
0.962306907762 0.626698723703
0.963252651189 0.60236654134
0.962028937444 0.678650423059


KeyboardInterrupt: 

In [19]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

KeyboardInterrupt: 

In [37]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [38]:
np.savez('./../data/features/results/emppca_lsaword_svc.npz', 
         score_lb=0.62990,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char SVC

In [29]:
X = StandardScaler().fit_transform(np.hstack((X_emp[:, 386:], X_emp_pca, X_lsa_char)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.981074213634 0.624642084559
0.981652012889 0.668066599169
0.979891710834 0.637541871735
0.981768261712 0.615696410248
0.980008877268 0.65216056139
0.982081323697 0.629610380685
0.981330340547 0.669160725852
0.981885216747 0.667098355447
0.9807528129 0.680382587349
0.981541924328 0.62895560082
---------------
[0.981198669456, 0.647331517725, 0.000724814879247, 0.0216241052085]


In [30]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [7]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [31]:
np.savez('./../data/features/results/_emppca_lsachar_svc.npz', 
         score_lb=0.62212771,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char RC on SVR

In [32]:
X = StandardScaler().fit_transform(np.hstack((X_emp[:, 386:], X_emp_pca, X_lsa_char)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.987769252582 0.671812800859
0.987656147159 0.684460853774
0.98754237893 0.692616954484
0.988793829448 0.626348636214
0.986693515821 0.68428517036
0.987773692341 0.692472264807
0.988116372826 0.662993522968
0.987549329881 0.677752392077
0.987208215631 0.699269720428
0.987435625131 0.66340750651
---------------
[0.987653835975, 0.675541982248, 0.000522778136415, 0.0201014219458]


In [33]:
model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [12]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [34]:
np.savez('./../data/features/results/_emppca_lsachar_rc_svr.npz', 
         score_lb=0.65015604,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_w2v_local SVC

In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp[:, 386:], X_emp_pca, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [18]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [19]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [20]:
np.savez('./../data/features/results/_emppca_w2vlocal_svc.npz', 
         score_lb=0.61530543,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_w2v_local RC on SVR

In [35]:
X = StandardScaler().fit_transform(np.hstack((X_emp[:, 386:], X_emp_pca, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.978439752226 0.663659081625
0.977644543011 0.658973039055
0.976961933637 0.649797425756
0.97775831124 0.677324265653
0.977936257003 0.682238396748
0.978277448905 0.645396471733
0.979473734882 0.664017858765
0.979021473634 0.679801661444
0.976974788135 0.66955531461
0.976747378635 0.682875565494
---------------
[0.977923562131, 0.667363908088, 0.000855205416522, 0.012676942266]


In [36]:
model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [24]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [37]:
np.savez('./../data/features/results/_emppca_w2vlocal_rc_svr.npz', 
         score_lb=0.63769467,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char + X_w2v_local SVC

In [38]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.981757195937 0.646793950146
0.981025241363 0.660580550508
0.98044990284 0.650290883573
0.981637281596 0.644923266841
0.983374632313 0.634579470842
0.981644645326 0.669248853592
0.980397525141 0.620609570457
0.981088017361 0.625696763108
0.980677319839 0.652064988218
0.980027483598 0.660918586624
---------------
[0.981207924531, 0.646570688391, 0.00091244020579, 0.0148886477657]


In [5]:
model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
y_pred = model.predict(X)

In [6]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [9]:
np.savez('./../data/features/results/emppca_lsachar_w2vlocal_svc.npz', 
         score_lb=0.62600,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char + X_w2v_local RC on SVR

In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [12]:
model = SVR(C=10, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [13]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [14]:
np.savez('./../data/features/results/emppca_lsachar_w2vlocal_rc_svr.npz', 
         score_lb=0.65211,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char + X_w2v_local RC on SVR - C 0.5

In [39]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=0.5, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.803113410167 0.635121064309
0.79971103293 0.67936329083
0.799824801159 0.678343778241
0.79971103293 0.669168164943
0.799379161612 0.683261783554
0.80188123556 0.652560179375
0.799229246168 0.662993522968
0.797093873716 0.695171181695
0.799595378215 0.685949469544
0.799595378215 0.696195816378
---------------
[0.799913455067, 0.673812825184, 0.0015201209736, 0.0182550084236]


In [20]:
model = SVR(C=0.5, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [21]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [22]:
np.savez('./../data/features/results/emppca_lsachar_w2vlocal_rc_svr_c05.npz', 
         score_lb=0.64532,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_pca + X_lsa_char + X_lsa_char + X_w2v_local RC on SVR - C 3

In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp[:, 386:], X_emp_pca, X_lsa_word, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=3, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [35]:
model = SVR(C=3, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [36]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [37]:
np.savez('./../data/features/results/emppca_lsaword_lsachar_w2vlocal_rc_svr_c3.npz', 
         score_lb=0.67260,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_good + X_lsa_word + X_lsa_char + X_w2v_local RC on SVR

In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp_good, X_lsa_word, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=3, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train] - 4.5)
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [ ]:
model = SVR(C=3, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all - 4.5)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [ ]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [ ]:
np.savez('./../data/features/results/empgood_lsaword_lsachar_w2vlocal_rc_svr_c3.npz', 
         score_lb=0.67534,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# X_emp_good poly2 + X_lsa_word + X_lsa_char + X_w2v_local RC on SVR

In [ ]:
X = StandardScaler().fit_transform(np.hstack((PolynomialFeatures(degree=2).fit_transform(X_emp_good), 
                                              X_lsa_word, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=3, kernel='rbf', degree=1, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train] - 4.5)
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

In [ ]:
model = SVR(C=3, kernel='rbf', degree=2, cache_size=4196*4)
model = model.fit(X[idx_row_train, :], y_train_all - 4.5)
rc = RegressionClassifier(model)
rc.fit(X, y_train_all)
y_pred = rc.predict(X)

In [ ]:
submission = pd.DataFrame({"id": df.index[df['median_relevance'].isnull()], 
                           "prediction": y_pred[idx_row_test].astype(int)})
submission.to_csv("submission.csv", index=False)

In [ ]:
np.savez('./../data/features/results/_empgoodpoly2_lsaword_lsachar_w2vlocal_rc_svr_c3.npz', 
         score_lb=0.67611959,
         scores_train_cv=np.array(scores), 
         scores_train_cv_mean=mean, 
         scores_train_cv_std=std, 
         y_pred=y_pred)

# TESTING

In [47]:
X = StandardScaler().fit_transform(np.hstack((X_emp_good, X_lsa_char, X_lsa_word, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVC(C=10, kernel='rbf', degree=2, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train])   
    y_train_pred = model.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = model.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print mean

0.960379254037 0.624965693126
0.958438945943 0.679194548656
0.95823798052 0.695371473622


KeyboardInterrupt: 

In [56]:
X = StandardScaler().fit_transform(np.hstack((PolynomialFeatures(degree=2).fit_transform(X_emp_good), 
                                              X_lsa_word, X_lsa_char, X_w2v_local)))
scores = []
for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    model = SVR(C=3, kernel='rbf', degree=1, cache_size=4196*4)
    model = model.fit(X[i_train, :], y_train_all[i_train] - 4.5)
    rc = RegressionClassifier(model)
    rc.fit(X, y_train_all)
    y_train_pred = rc.predict(X[i_train, :])
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(X[i_cv, :])
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

0.917115353543 0.689139454229
0.917574918124 0.684460853774
0.916323467606 0.714026718849
0.917233613437 0.724221844736
0.917090367809 0.685308557166


KeyboardInterrupt: 

In [48]:
X = StandardScaler().fit_transform(np.hstack((X_emp_good, X_lsa_word, X_lsa_char, X_w2v_local)))
scores = []

num_round = 100
param = {'objective': 'reg:linear',
         'nthread': 8,
         'eval_metric': 'rmse',
         'subsample': 0.8,
         'silent': 1}
plst = param.items()

for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    
    dtrain = xgb.DMatrix(X[i_train, :], label=y_train_all[i_train])
    dcv = xgb.DMatrix(X[i_cv, :], label=y_train_all[i_cv])
    evallist = [(dcv, 'eval')]
    
    bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=5)
    
    rc = RegressionClassifier(bst)
    rc.fit(None, y_train_all)
    y_train_pred = rc.predict(dtrain)
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(dcv)
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '[%s]' % ', '.join(map(str, np.hstack((mean, std)).tolist()))

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.146998
[1]	eval-rmse:1.589428
[2]	eval-rmse:1.226997
[3]	eval-rmse:1.000726
[4]	eval-rmse:0.873514
[5]	eval-rmse:0.800046
[6]	eval-rmse:0.758873
[7]	eval-rmse:0.738359
[8]	eval-rmse:0.730486
[9]	eval-rmse:0.722334
[10]	eval-rmse:0.720635
[11]	eval-rmse:0.718853
[12]	eval-rmse:0.717779
[13]	eval-rmse:0.718563
[14]	eval-rmse:0.718861
[15]	eval-rmse:0.717563
[16]	eval-rmse:0.717160
[17]	eval-rmse:0.715467
[18]	eval-rmse:0.714808
[19]	eval-rmse:0.715038
[20]	eval-rmse:0.717068
[21]	eval-rmse:0.716013
[22]	eval-rmse:0.715515
[23]	eval-rmse:0.717944
Stopping. Best iteration:
[18]	eval-rmse:0.714808

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.147975
[1]	eval-rmse:1.595110
[2]	eval-rmse:1.229753
[3]	eval-rmse:1.003925
[4]	eval-rmse:0.870494
[5]	eval-rmse:0.797326
[6]	eval-rmse:0.757405
[7]	eval-rmse:0.739400
[8]	eval-rmse:0.731534
[9]	eval-rmse:0.726585
[10]	eval-rmse:0.721193
[11]	eval-rms

0.865234229612 0.661620651817
0.838733535461

[16]	eval-rmse:0.725432
Stopping. Best iteration:
[11]	eval-rmse:0.720567

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.153580
[1]	eval-rmse:1.598267
[2]	eval-rmse:1.230134
[3]	eval-rmse:0.996911
[4]	eval-rmse:0.856710
[5]	eval-rmse:0.779129
[6]	eval-rmse:0.734178
[7]	eval-rmse:0.710256
[8]	eval-rmse:0.702134
[9]	eval-rmse:0.698365
[10]	eval-rmse:0.695617
[11]	eval-rmse:0.694606
[12]	eval-rmse:0.693922
[13]	eval-rmse:0.691066
[14]	eval-rmse:0.690955
[15]	eval-rmse:0.690659
[16]	eval-rmse:0.690516
[17]	eval-rmse:0.691145
[18]	eval-rmse:0.691725
[19]	eval-rmse:0.692683
[20]	eval-rmse:0.692345
[21]	eval-rmse:0.693454


 0.651836450933
0.863990082288

Stopping. Best iteration:
[16]	eval-rmse:0.690516

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.147801
[1]	eval-rmse:1.587214
[2]	eval-rmse:1.222159
[3]	eval-rmse:0.989749
[4]	eval-rmse:0.850875
[5]	eval-rmse:0.772732
[6]	eval-rmse:0.727401
[7]	eval-rmse:0.704215
[8]	eval-rmse:0.691692
[9]	eval-rmse:0.686941
[10]	eval-rmse:0.681840
[11]	eval-rmse:0.681311
[12]	eval-rmse:0.681716
[13]	eval-rmse:0.682588
[14]	eval-rmse:0.682499
[15]	eval-rmse:0.681181
[16]	eval-rmse:0.681578
[17]	eval-rmse:0.679843
[18]	eval-rmse:0.678927
[19]	eval-rmse:0.679389
[20]	eval-rmse:0.682441
[21]	eval-rmse:0.683538
[22]	eval-rmse:0.681429
[23]	eval-rmse:0.684007


...

[20]	eval-rmse:0.675350
Stopping. Best iteration:
[15]	eval-rmse:0.670293

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.154510
[1]	eval-rmse:1.601201
[2]	eval-rmse:1.235402
[3]	eval-rmse:1.011742
[4]	eval-rmse:0.878774
[5]	eval-rmse:0.804458
[6]	eval-rmse:0.765781
[7]	eval-rmse:0.747751
[8]	eval-rmse:0.735395
[9]	eval-rmse:0.729205
[10]	eval-rmse:0.725937
[11]	eval-rmse:0.725609
[12]	eval-rmse:0.725798
[13]	eval-rmse:0.723233
[14]	eval-rmse:0.726639
[15]	eval-rmse:0.727668
[16]	eval-rmse:0.725667
[17]	eval-rmse:0.723917


 0.700294355112
0.853377724941

[18]	eval-rmse:0.724932
Stopping. Best iteration:
[13]	eval-rmse:0.723233

Will train until eval error hasn't decreased in 5 rounds.
[0]	eval-rmse:2.147774
[1]	eval-rmse:1.586430
[2]	eval-rmse:1.219696
[3]	eval-rmse:0.991563
[4]	eval-rmse:0.858700
[5]	eval-rmse:0.783849
[6]	eval-rmse:0.739475
[7]	eval-rmse:0.716691
[8]	eval-rmse:0.707886
[9]	eval-rmse:0.705403
[10]	eval-rmse:0.703345
[11]	eval-rmse:0.701050
[12]	eval-rmse:0.701219
[13]	eval-rmse:0.701890
[14]	eval-rmse:0.704360
[15]	eval-rmse:0.703719


 0.652136524993
0.843258002196 0.689023373595
[0.853120885666, 0.679624210917, 0.010187073537, 0.0184739091085]


[16]	eval-rmse:0.704601
Stopping. Best iteration:
[11]	eval-rmse:0.701050



In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_word)))
scores = []

num_round = 100
param = {'objective': 'multi:softmax',
         'nthread': 8,
         'eval_metric': 'merror',
         'subsample': 0.8,
         'num_class': 4,
         'silent': 1}
plst = param.items()

for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    
    dtrain = xgb.DMatrix(X[i_train, :], label=y_train_all[i_train] - 1)
    dcv = xgb.DMatrix(X[i_cv, :], label=y_train_all[i_cv] - 1)
    evallist = [(dcv, 'eval')]
    
    bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=5)
    
    y_train_pred = bst.predict(dtrain) + 1
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = bst.predict(dcv) + 1
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print mean

In [ ]:
X = StandardScaler().fit_transform(np.hstack((X_emp_pca, X_lsa_word)))
scores = []

num_round = 100
param = {'objective': 'reg:linear',
         'nthread': 8,
         'eval_metric': 'rmse',
         'subsample': 0.8,
         'silent': 1}
plst = param.items()

rc_best = None
score_cv_best = -1

for i_train, i_cv in StratifiedKFold(y_train_all, 10, shuffle=True):
    
    dtrain = xgb.DMatrix(X[i_train, :], label=y_train_all[i_train])
    dcv = xgb.DMatrix(X[i_cv, :], label=y_train_all[i_cv])
    evallist = [(dcv, 'eval')]
    
    bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=5)
    
    rc = RegressionClassifier(bst)
    rc.fit(None, y_train_all)
    y_train_pred = rc.predict(dtrain)
    score_train = quadratic_weighted_kappa(y_train_all[i_train], y_train_pred)
    y_cv_pred = rc.predict(dcv)
    score_cv = quadratic_weighted_kappa(y_train_all[i_cv], y_cv_pred)
    print score_train, score_cv
    scores.append((score_train, score_cv))
    
    if score_cv > score_cv_best:        
        score_cv_best = score_cv
        rc_best = rc

mean = np.array(scores).mean(axis=0)
std = np.array(scores).std(axis=0)
print '---------------'
print mean

y_pred = rc_best.predict(xgb.DMatrix(X, label=y_train_all))